### Training and Validation of YOLOv8x Model for New Dataset
#### File Set-Up

In [ ]:
from PIL import Image
import numpy as np
import pandas as pd

In [ ]:
# solves occasional error with Linux commands and encoding
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
# mount google drive for saving runs and model files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# copy dataset from google drive
!unzip /content/drive/MyDrive/UniStuff/Dissertation/Dataset_zips/WSP-9.zip -d /content/datasets/

In [ ]:
# install and import ultralytics

!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Setup complete ✅ (12 CPUs, 53.0 GB RAM, 32.4/78.2 GB disk)


### Model Training

In [ ]:
# train model on new dataset, 100 epochs, early stopping 10 epochs
model_new_dataset = YOLO("yolov8x.pt")

results = model_new_dataset.train(data="/content/datasets/WSP-9/data.yaml", project="/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns", 
                                       name="model_new_dataset", epochs=100, patience=10, plots=True)

100%|██████████| 131M/131M [00:00<00:00, 475MB/s] 

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8x.yaml, data=/content/datasets/WSP-9/data.yaml, epochs=100, patience=10, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns, name=model_expanded_dataset, exist_ok=False, pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False

In [ ]:
# resume stopped training
model_new_dataset = YOLO("/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns/model_new_dataset2/weights/last.pt")
results = model_new_dataset.train(resume=True)

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
yolo/engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns/model_expanded_dataset2/weights/last.pt, data=/content/datasets/WSP-9/data.yaml, epochs=100, patience=10, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns, name=model_expanded_dataset, exist_ok=False, pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, 

### Model Validation

In [ ]:
# validate new model
model_new_dataset_best = YOLO("/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns/model_new_dataset2/weights/best.pt")
  validation = model_new_dataset_best.val(plots=True, save_json=True)

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 268 layers, 68132235 parameters, 0 gradients, 257.4 GFLOPs
val: Scanning /content/datasets/WSP-9/valid/labels.cache... 359 images, 14 backgrounds, 0 corrupt: 100%|██████████| 359/359 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/23 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input,

In [ ]:
# save validation files to drive
!cp -r /content/runs/detect/val /content/drive/MyDrive/UniStuff/Dissertation/ModelRuns/model_expanded_dataset2/